In [9]:
import numpy as np
from collections import defaultdict
import random

In [17]:
class TicTacToeMC:
    def __init__(self, player='X'):
        self.player = player
        self.states = defaultdict(float)  # state-action value table
        self.returns = defaultdict(list)  # to store returns for averaging
        self.gamma = 1.0  # discount factor
        self.epsilon = 0.5  # exploration rate, can decay over time

    def reset_board(self):
        return np.zeros((3, 3), dtype=str)
    
    def get_available_actions(self, board):
        return [(i, j) for i in range(3) for j in range(3) if board[i, j] == '']
    
    def is_winner(self, board, player):
        for i in range(3):
            if np.all(board[i, :] == player) or np.all(board[:, i] == player):
                return True
        if board[0, 0] == board[1, 1] == board[2, 2] == player or \
           board[0, 2] == board[1, 1] == board[2, 0] == player:
            return True
        return False
    
    def is_draw(self, board):
        return '' not in board

    def update_board(self, board, action, player):
        new_board = board.copy()
        new_board[action] = player
        return new_board
        
    def display_board(self, board):
        print("\n".join([" ".join([cell if cell != '' else '.' for cell in row]) for row in board]))
    
    
    def get_state_key(self, board):
        # Convert each row to a tuple and then the whole board to a tuple of tuples
        return tuple(tuple(row) for row in board)

    def choose_action(self, board):
        actions = self.get_available_actions(board)
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(actions)
        else:
            # Pick the action with the highest estimated reward
            state_key = self.get_state_key(board)
            q_values = {action: self.states[(state_key, action)] for action in actions}
            return max(q_values, key=q_values.get, default=random.choice(actions))

    def generate_episode(self):
        board = self.reset_board()
        episode = []
        current_player = self.player
        while True:
            action = self.choose_action(board)
            episode.append((self.get_state_key(board), action, current_player))
            board = self.update_board(board, action, current_player)
            if self.is_winner(board, current_player):
                return episode, 1, current_player  # Win reward
            elif self.is_draw(board):
                return episode, 0, None  # Draw reward
            current_player = 'O' if current_player == 'X' else 'X'

    def update_policy(self, episode, reward, winner):
        # Using first-visit Monte Carlo control
        G = reward  # Reward for terminal state
        for i, (state_key, action, player) in enumerate(episode):
            if (state_key, action) not in episode[:i]:  # Check first visit
                k = 1 if winner == player else -1
                self.returns[(state_key, action)].append(G*k)
                self.states[(state_key, action)] = np.mean(self.returns[(state_key, action)])

    def train(self, episodes=10000, decay_rate=1):
        for episode_num in range(episodes):
            episode, reward, winner = self.generate_episode()
            self.update_policy(episode, reward, winner)
            # Decay epsilon to reduce exploration over time
            self.epsilon *= decay_rate
            
            # Optionally, print progress every 1000 episodes
            if episode_num % 1000 == 0:
                print(episode_num)
                for i in range(3):
                    for j in range(3):
                        print((i,j),agent.states[(agent.get_state_key(np.zeros((3,3),dtype=str)),(i,j))])

    def get_best_policy(self, board):
        actions = self.get_available_actions(board)
        state_key = self.get_state_key(board)
        q_values = {action: self.states[(state_key, action)] for action in actions}
        best_action = max(q_values, key=q_values.get, default=None)
        return best_action
        
    def play(self):
        board = self.reset_board()
        current_player = 'X'
        
        while True:
            self.display_board(board)
            if current_player == 'X':
                action = self.get_best_policy(board)
                if action is None:
                    print("No valid action found. Game over.")
                    break
                print(f"Agent 'X' chooses {action}")
            else:
                action = tuple(map(int, input("Enter your move (row col): ").split()))
            
            board = self.update_board(board, action, current_player)
            if self.is_winner(board, current_player):
                self.display_board(board)
                print(f"Player '{current_player}' wins!")
                break
            elif self.is_draw(board):
                self.display_board(board)
                print("It's a draw!")
                break
            
            current_player = 'O' if current_player == 'X' else 'X'

In [18]:
# Example: Using the trained agent
agent = TicTacToeMC()

In [19]:
agent.train(episodes=10000)

0
(0, 0) 1.0
(0, 1) 0.0
(0, 2) 0.0
(1, 0) 0.0
(1, 1) 0.0
(1, 2) 0.0
(2, 0) 0.0
(2, 1) 0.0
(2, 2) 0.0
1000
(0, 0) 0.3212996389891697
(0, 1) -0.14545454545454545
(0, 2) 0.30985915492957744
(1, 0) 0.23469387755102042
(1, 1) 0.23809523809523808
(1, 2) 0.1276595744680851
(2, 0) 0.15873015873015872
(2, 1) 0.29850746268656714
(2, 2) 0.3
2000
(0, 0) 0.26737967914438504
(0, 1) 0.018018018018018018
(0, 2) 0.24113475177304963
(1, 0) 0.24358974358974358
(1, 1) 0.31176470588235294
(1, 2) 0.08163265306122448
(2, 0) 0.17543859649122806
(2, 1) 0.2047244094488189
(2, 2) 0.25595238095238093
3000
(0, 0) 0.2493857493857494
(0, 1) 0.08588957055214724
(0, 2) 0.211340206185567
(1, 0) 0.2315270935960591
(1, 1) 0.34630872483221475
(1, 2) 0.026143790849673203
(2, 0) 0.17105263157894737
(2, 1) 0.13690476190476192
(2, 2) 0.2371638141809291
4000
(0, 0) 0.2502818489289741
(0, 1) 0.06944444444444445
(0, 2) 0.2190082644628099
(1, 0) 0.1732283464566929
(1, 1) 0.3756777691711851
(1, 2) 0.05714285714285714
(2, 0) 0.2175

In [22]:
agent.play()

. . .
. . .
. . .
Agent 'X' chooses (1, 1)
. . .
. X .
. . .


Enter your move (row col):  0 0


O . .
. X .
. . .
Agent 'X' chooses (0, 1)
O X .
. X .
. . .


Enter your move (row col):  2 1


O X .
. X .
. O .
Agent 'X' chooses (1, 0)
O X .
X X .
. O .


Enter your move (row col):  1 2


O X .
X X O
. O .
Agent 'X' chooses (0, 2)
O X X
X X O
. O .


Enter your move (row col):  2 0


O X X
X X O
O O .
Agent 'X' chooses (2, 2)
O X X
X X O
O O X
It's a draw!
